In [ ]:
# ! pip3 install kfp==1.8.10
# ! pip3 install tfx==1.5.0
# ! pip3 install pyparsing==2.4.2

In [1]:
import tensorflow as tf
import tfx
import os

In [2]:
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://text-analysis-323506-artifact-store'
ENDPOINT = 'https://1b7bb986cc987470-dot-us-central1.pipelines.googleusercontent.com'
CUSTOM_SERVICE_ACCOUNT = 'my-api-sa@text-analysis-323506.iam.gserviceaccount.com'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [3]:
os.environ['GCP_REGION'] = GCP_REGION
os.environ['ARTIFACT_STORE_URI'] = ARTIFACT_STORE_URI
os.environ['CUSTOM_SERVICE_ACCOUNT'] = CUSTOM_SERVICE_ACCOUNT
os.environ['PROJECT_ID'] = PROJECT_ID

In [4]:
PIPELINE_NAME = 'tfx_covertype_training'
MODEL_NAME = 'tfx_covertype_classifier'
DATA_ROOT_URI = 'gs://text-analysis-323506/covertype'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)
RUNTIME_VERSION = '2.6'
PYTHON_VERSION = '3.7'
USE_KFP_SA=False
ENABLE_TUNING=False

In [5]:
os.environ['PIPELINE_NAME'] = PIPELINE_NAME
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['DATA_ROOT_URI'] = DATA_ROOT_URI
os.environ['CUSTOM_TFX_IMAGE'] = CUSTOM_TFX_IMAGE
os.environ['RUNTIME_VERSION'] = RUNTIME_VERSION
os.environ['PYTHON_VERSION'] = PYTHON_VERSION
os.environ['USE_KFP_SA'] = str(USE_KFP_SA)
os.environ['ENABLE_TUNING'] = str(ENABLE_TUNING)

In [6]:
%cd pipeline

/home/jupyter/TFX-Pipelines-on-Google-Cloud/pipeline


In [7]:
! tfx pipeline compile --engine kubeflow --pipeline_path runner.py

CLI
Compiling pipeline
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying features.py -> build/lib
copying config.py -> build/lib
copying model.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
installing to /tmp/tmpczv6m_gz
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpczv6m_gz
copying build/lib/features.py -> /tmp/tmpczv6m_gz
copying build/lib/pipeline.py -> /tmp/tmpczv6m_gz
copying build/lib/config.py -> /tmp/tmpczv6m_gz
copying build/lib/model.py -> /tmp/tmpczv6m_gz
copying build/lib/runner.py -> /tmp/tmpczv6m_gz
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_c

In [9]:
!tfx pipeline create --pipeline_path=runner.py --endpoint={ENDPOINT}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying features.py -> build/lib
copying config.py -> build/lib
copying model.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
installing to /tmp/tmpv3faz_ji
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpv3faz_ji
copying build/lib/features.py -> /tmp/tmpv3faz_ji
copying build/lib/pipeline.py -> /tmp/tmpv3faz_ji
copying build/lib/config.py -> /tmp/tmpv3faz_ji
copying build/lib/model.py -> /tmp/tmpv3faz_ji
copying build/lib/runner.py -> /tmp/tmpv3faz_ji
running install_egg

#### Pipeline runs can be created either through front-end UI or also thorugh tfx command

In [12]:
! tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: tfx_covertype_training
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: tfx_covertype_training
+========================+======================================+========+===========================+==============================================================================================================================+
| pipeline_name          | run_id                               | status | created_at                | link                                                                                                                         |
+========================+======================================+========+===========================+==============================================================================================================================+
| tfx_covertype_training | 14058537-8361-4503-a5d7-4985dfabbbc7 | None   | 2021-12-18T18:02:05+00:00 | https://1b7bb986cc

In [15]:
! tfx run list --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

CLI
Listing all runs of pipeline: tfx_covertype_training
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
+========================+======================================+=========+===========================+==============================================================================================================================+
| pipeline_name          | run_id                               | status  | created_at                | link                                                                                                                         |
+========================+======================================+=========+===========================+==============================================================================================================================+
| tfx_covertype_training | 1e40d136-8acf-4e80-8c46-3997ac1ee13f | Running | 2021-12-18T17:42:11+00:00 | https://1b7bb986cc987470-dot-us-central1.pipelines.googleuserc

In [14]:
RUN_ID='1e40d136-8acf-4e80-8c46-3997ac1ee13f'

! tfx run status --pipeline_name {PIPELINE_NAME} --run_id {RUN_ID} --endpoint {ENDPOINT}

CLI
Retrieving run status.
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
+========================+======================================+=========+===========================+==============================================================================================================================+
| pipeline_name          | run_id                               | status  | created_at                | link                                                                                                                         |
+========================+======================================+=========+===========================+==============================================================================================================================+
| tfx_covertype_training | 1e40d136-8acf-4e80-8c46-3997ac1ee13f | Running | 2021-12-18T17:42:11+00:00 | https://1b7bb986cc987470-dot-us-central1.pipelines.googleusercontent.com/#/runs/details/1e40

#### Creating a pipeline run with tuning

In [16]:
os.environ['ENABLE_TUNING'] = str(True)

In [17]:
! tfx pipeline compile --engine kubeflow --pipeline_path runner.py

CLI
Compiling pipeline
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying features.py -> build/lib
copying config.py -> build/lib
copying model.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
installing to /tmp/tmpc7u67ar8
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpc7u67ar8
copying build/lib/features.py -> /tmp/tmpc7u67ar8
copying build/lib/pipeline.py -> /tmp/tmpc7u67ar8
copying build/lib/config.py -> /tmp/tmpc7u67ar8
copying build/lib/model.py -> /tmp/tmpc7u67ar8
copying build/lib/runner.py -> /tmp/tmpc7u67ar8
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_c

In [18]:
! tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying features.py -> build/lib
copying config.py -> build/lib
copying model.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
installing to /tmp/tmpj8gpear9
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpj8gpear9
copying build/lib/features.py -> /tmp/tmpj8gpear9
copying build/lib/pipeline.py -> /tmp/tmpj8gpear9
copying build/lib/config.py -> /tmp/tmpj8gpear9
copying build/lib/model.py -> /tmp/tmpj8gpear9
copying build/lib/runner.py -> /tmp/tmpj8gpear9
running install_egg

In [19]:
! tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: tfx_covertype_training
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: tfx_covertype_training
+========================+======================================+========+===========================+==============================================================================================================================+
| pipeline_name          | run_id                               | status | created_at                | link                                                                                                                         |
+========================+======================================+========+===========================+==============================================================================================================================+
| tfx_covertype_training | a6ffda91-4c63-4ea8-a1f4-34845d92c62d | None   | 2021-12-18T18:18:25+00:00 | https://1b7bb986cc